## Part 0: Data Preprocessing

This section of the code fetches the CIFAR-10 dataset using the torchvision library and applies a series of transformations to preprocess the data. Here's a breakdown of the preprocessing steps:

1. Data Fetching:

- The CIFAR-10 dataset is downloaded and loaded using `torchvision.datasets.CIFAR10`.

2. Data Transformations:

- `ToTensor`: Converts images to PyTorch tensors
- `Normalize`: Normalizes the pixel values of each image. The pixel values are scaled to a range of [-1, 1] by subtracting the mean (0.5, 0.5, 0.5) and dividing by the standard deviation (0.5, 0.5, 0.5) for each color channel (RGB).

3. Dataset Splitting:

- The dataset is split into three subsets: training, validation, and testing.
- The proportions are set as 80% for training, 10% for validation, and 10% for testing.
- The `torch.utils.data.random_split` function is used for splitting.

4. Data Loaders:

- `DataLoader`: Wraps the datasets into iterable loaders, allowing batches of data to be efficiently fetched during training and evaluation.
- Parameters:
  - `batch_size=64`: The data is processed in mini-batches of 64 images to optimize memory usage and computational efficiency.
  - `shuffle=True` (for training): Randomly shuffles the training data to reduce model bias.
  - `shuffle=False` (for validation and testing): Maintains the order for consistent evaluation.
  

In [7]:
import torch
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, random_split

transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,0.5,0.5),(0.5,0.5,0.5))
])

dataset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)

train_size = int(0.8 * len(dataset))
val_size = int(0.1 * len(dataset))
test_size = len(dataset) - train_size - val_size

train_data, val_data, test_data = random_split(dataset, [train_size, val_size, test_size])

train_loader = DataLoader(train_data, batch_size=64, shuffle=True)
val_loader = DataLoader(val_data, batch_size=64, shuffle=False)
test_loader = DataLoader(test_data, batch_size=64, shuffle=False)

data_iter = iter(train_loader)
images, labels = next(data_iter)

print({'images': images.shape, 'labels': labels.shape})

Files already downloaded and verified
{'images': torch.Size([64, 3, 32, 32]), 'labels': torch.Size([64])}


## Part 1: MLP Model Architecture

This section defines, trains, and evaluates a simple Multi-Layer Perceptron (MLP) model for classifying CIFAR-10 images.

1. **Model Definition**:
   - The `MLPClassifier` class defines the architecture:
     - Three fully connected layers: 
       - `fc1`: Input layer maps 3 * 32 * 32 (32 x 32 images with 3 color channels) = 3072 pixels to 512 neurons.
       - `fc2`: Hidden layer with 128 neurons.
       - `fc3`: Output layer with 10 neurons (for the 10 classes).
     - ReLU activation is used after each layer, except the last one.

2. **Loss and Optimizer**:
   - Loss: `CrossEntropyLoss`, suitable for multi-class classification.
   - Optimizer: `Adam` optimizer with a learning rate of 0.001.

3. **Training Loop**:
   - The model is trained for 25 epochs.
   - Steps:
     - Set the model to training mode (`model.train()`).
     - Loop through batches of images and labels from `train_loader`.
     - Compute predictions, loss, and gradients, and update weights.
     - Track the running loss for monitoring.

4. **Validation**:
   - After training, the model is evaluated on the validation set (`model.eval()`).
   - Steps:
     - Loop through batches from `val_loader` without gradient computation (`torch.no_grad()`).
     - Compute predictions and compare them with ground-truth labels.
     - Calculate and print the accuracy.


In [25]:
import torch.nn as nn
import torch.optim as optim
from torch import Tensor

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

class MLPClassifier(nn.Module):
    def __init__(self):
        super(MLPClassifier, self).__init__()
        self.fc1 = nn.Linear(3*32*32, 512)
        self.fc2 = nn.Linear(512, 128)
        self.fc3 = nn.Linear(128, 10)
        self.relu = nn.ReLU()
    
    def forward(self, x: Tensor) -> Tensor:
        x = x.view(x.size(0), -1)
        x = self.relu(self.fc1(x))
        x = self.relu(self.fc2(x))
        x = self.fc3(x)
        return x
    
model = MLPClassifier().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

num_epochs = 25
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    print(f"Epoch {epoch+1}, loss: {running_loss / len(train_loader)}")

model.eval()
correct, total = 0, 0
with torch.no_grad():
    for images, labels in val_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f"Validation accuracy: {100 * correct / total}")

Epoch 1, loss: 1.6667456855773926
Epoch 2, loss: 1.4602145307540892
Epoch 3, loss: 1.3486177486419677
Epoch 4, loss: 1.2611588106155396
Epoch 5, loss: 1.1833021311759948
Epoch 6, loss: 1.0962547177314759
Epoch 7, loss: 1.0235486729621888
Epoch 8, loss: 0.9549388641357421
Epoch 9, loss: 0.8759211818695068
Epoch 10, loss: 0.8144184517860412
Epoch 11, loss: 0.7524882660388946
Epoch 12, loss: 0.6988078862190247
Epoch 13, loss: 0.6388286056041718
Epoch 14, loss: 0.5990701889991761
Epoch 15, loss: 0.5516770469903945
Epoch 16, loss: 0.5126571859121323
Epoch 17, loss: 0.480721221280098
Epoch 18, loss: 0.44483970971107484
Epoch 19, loss: 0.41846851511001587
Epoch 20, loss: 0.3900272782087326
Epoch 21, loss: 0.37842908816337584
Epoch 22, loss: 0.35487020299434663
Epoch 23, loss: 0.347210074532032
Epoch 24, loss: 0.32436544207334517
Epoch 25, loss: 0.3127782540082932
Validation accuracy: 51.34


## Part 2: CNN Model Architecture

This section defines a Convolutional Neural Network (CNN) for classifying CIFAR-10 images. The CNN is designed to leverage convolutional layers for feature extraction.

1. **Model Definition**:
   - The `CNNClassifier` class defines the architecture:
     - **Convolutional Layers**:
       - `conv1`: Convolution with 32 filters, (3 * 3) kernel, stride 1, and padding 1.
       - `conv2`: Convolution with 64 filters, (3 * 3) kernel, stride 1, and padding 1.
       - `conv3`: Convolution with 128 filters, (3 * 3) kernel, stride 1, and padding 1.
     - **Pooling**:
       - Max-pooling (2 * 2), stride 2 is applied after each convolution layer to downsample the feature maps.
     - **Fully Connected Layers**:
       - `fc1`: Linear layer mapping 128 * 4 * 4 = 2048 features to 256.
       - `fc2`: Output layer with 10 neurons (one for each class).
     - Additional features:
       - ReLU activation after each layer (except the last).
       - Dropout (50%) in the fully connected layers to prevent overfitting.

2. **Training and Evaluation**:
   - The training and evaluation processes are the same as for the MLP classifier, utilizing the `CrossEntropyLoss` and `Adam` optimizer.


In [9]:
class CNNClassifier(nn.Module):
    def __init__(self):
        super(CNNClassifier, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=32, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, stride=1, padding=1)
        self.conv3 = nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3, stride=1, padding=1)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.fc1 = nn.Linear(128 * 4 * 4, 256)
        self.fc2 = nn.Linear(256, 10)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(p=0.5)
    
    def forward(self, x : Tensor) -> Tensor:
        x = self.relu(self.conv1(x))
        x = self.pool(x)
        x = self.relu(self.conv2(x))
        x = self.pool(x)
        x = self.relu(self.conv3(x))
        x = self.pool(x)
        x = x.view(x.size(0), -1)
        x = self.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.fc2(x)
        return x
    
model = CNNClassifier().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

num_epochs = 25
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    print(f"Epoch {epoch+1}, loss: {running_loss / len(train_loader)}")

model.eval()
correct, total = 0, 0
with torch.no_grad():
    for images, labels in val_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f"Validation accuracy: {100 * correct / total}")
    

Epoch 1, loss: 1.5425277339935304
Epoch 2, loss: 1.154657793045044
Epoch 3, loss: 0.972494538974762
Epoch 4, loss: 0.8484634868621826
Epoch 5, loss: 0.7580944016456604
Epoch 6, loss: 0.6799252791881562
Epoch 7, loss: 0.6099780755519867
Epoch 8, loss: 0.5509095715999603
Epoch 9, loss: 0.4976423180341721
Epoch 10, loss: 0.46005331478118894
Epoch 11, loss: 0.4129638736486435
Epoch 12, loss: 0.37255605022907257
Epoch 13, loss: 0.3487615005493164
Epoch 14, loss: 0.3183569635391235
Epoch 15, loss: 0.29728884640932085
Epoch 16, loss: 0.27631196895837784
Epoch 17, loss: 0.2577621481895447
Epoch 18, loss: 0.2396953366756439
Epoch 19, loss: 0.2263425535529852
Epoch 20, loss: 0.21669124541282653
Epoch 21, loss: 0.20572140931487085
Epoch 22, loss: 0.19801197879314422
Epoch 23, loss: 0.18922382845282554
Epoch 24, loss: 0.18619295721054077
Epoch 25, loss: 0.17833629528880118
Validation accuracy: 75.82
